In [1]:
import pandas as pd

In [2]:
events = pd.read_pickle('../events_agg.pkl')
session = pd.read_pickle('../session_agg.pkl')
atr = pd.read_pickle('../attributes_agg.pkl')
user_date = pd.read_pickle('../user_date.pkl')

In [3]:
df = session

In [4]:
user_date = user_date.groupby('user_id_hash').user_created_timestamp.max()

In [5]:
user_date = user_date.to_frame()

In [6]:
user_date.reset_index(inplace=True)

In [7]:
df = df.merge(user_date, on=['user_id_hash'], how='left')

In [8]:
df.head()

,week_start,user_id_hash,no_city,city,is_user_first_session,country,country_count,region,region_count,locale,os_name,user_days,total_session_time,mean_session_time,median_session_time,user_created_timestamp
0,2018-10-01,000062e9be78f3da274fec338e78f89d12000e781967f2...,5,campbell,True,US,1,ca,1,en_US,Android OS,1,4730609,946121.8,1176899.0,2018-10-07
1,2018-10-01,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,douglasville,True,US,1,ga,1,en_US,Android OS,2,403432,201716.0,201716.0,2018-10-05
2,2018-10-01,0003f8bda56230a49445880e559b718e94ba37344494a7...,2,middlesboro,True,US,1,ky,1,en_US,Android OS,2,150138,75069.0,75069.0,2018-10-03
3,2018-10-01,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1,omaha,True,US,1,ne,1,en_US,Android OS,1,0,0.0,0.0,2018-10-02
4,2018-10-01,00059c820dca1ccadfd45361c40f68b656df8a5212b993...,1,rio de janeiro,False,BR,1,rj,1,pt-BR_BR,iOS,1,123131,123131.0,123131.0,2018-10-01


In [9]:
purchase = pd.read_pickle('../purchase.pkl')

In [10]:
purchase.reset_index(inplace=True, drop=True)

In [11]:
t1 = purchase.groupby(['week_start', 'user_id_hash']).event_value.count()\
            .reset_index().rename(columns={'event_value': 'no_of_buys'})
t2 = purchase.groupby(['week_start', 'user_id_hash']).event_value.sum()\
            .reset_index().rename(columns={'event_value': 'sum_of_buys'})

In [12]:
purchase = t1.merge(t2, on=['week_start', 'user_id_hash'])

In [13]:
purchase.head()

,week_start,user_id_hash,no_of_buys,sum_of_buys
0,2018-10-01,001c2d06972b17ee6a075ff9fbdae33eccb0bbf28357ea...,1,1.393000
1,2018-10-01,0026497d2043c8530220eeb8e66051982cc6df3139a9b0...,3,10.478999
2,2018-10-01,006b49314baf054c636e7448d4f9b0df046aae9e9c17d0...,10,20.229999
3,2018-10-01,007327aa3a27718c76e6cd73c3971304280c8856f62940...,1,1.393000
4,2018-10-01,0088d9be17b2ec4b3621150117a0ede815547d4241eae1...,2,6.986000


In [14]:
import datetime as dt

In [15]:
purchase1 = purchase.copy(deep=True)
purchase1.week_start = purchase1.week_start.apply(lambda x: x- dt.timedelta(days=7))

In [16]:
purchase.sort_values('week_start', inplace=True)
purchase1.sort_values('week_start', inplace=True)

In [17]:
purchase2 = purchase.copy(deep=True)
purchase2.week_start = purchase2.week_start.apply(lambda x: x - dt.timedelta(days=14))

In [18]:
df.week_start = df.week_start.astype('category')

In [19]:
df = df.merge(purchase, on=['user_id_hash', 'week_start'], how='left')
df.no_of_buys.fillna(0, inplace=True)
df.sum_of_buys.fillna(0, inplace=True)

In [20]:
df = df.merge(purchase1, on=['user_id_hash', 'week_start'], how='left')
df.drop(['sum_of_buys_y'], inplace=True, axis=1)
df.rename(columns={'no_of_buys_x': 'no_of_buys',
                    'sum_of_buys_x': 'sum_of_buys',
                    'no_of_buys_y': 'y1'}, inplace=True)

In [23]:
df.week_start = df.week_start.astype('category')

In [24]:
df = df.merge(purchase2, on=['user_id_hash', 'week_start'], how='left')
df.drop(['sum_of_buys_y'], inplace=True, axis=1)
df.rename(columns={'no_of_buys_x': 'no_of_buys',
                    'sum_of_buys_x': 'sum_of_buys',
                    'no_of_buys_y': 'y2'}, inplace=True)

In [25]:
df['y1'].fillna(0, inplace=True)
df['y2'].fillna(0, inplace=True)
df['y2'] = df['y1'] + df['y2']
df['y1'] = df['y1'] > 0
df['y2'] = df['y2'] > 0

In [26]:
df.isna().sum(axis=0)

week_start                  0
user_id_hash                0
no_city                     0
city                      389
is_user_first_session       0
country                   946
country_count               0
region                    394
region_count                0
locale                      0
os_name                     0
user_days                   0
total_session_time          0
mean_session_time           0
median_session_time         0
user_created_timestamp      0
no_of_buys                  0
sum_of_buys                 0
y1                          0
y2                          0
dtype: int64

In [27]:
sum(df[df.country.isna()]['y1'])

7

In [28]:
sum(df[df.country.isna()]['y2'])

8

In [29]:
sum(df[df.city.isna()]['y1'])

2

In [30]:
sum(df[df.city.isna()]['y2'])

2

In [31]:
df.shape

(995976, 20)

In [35]:
df = df[~df.city.isna()]
df = df[~df.country.isna()]
df = df[~df.region.isna()]

In [36]:
df.shape

(994636, 20)

In [37]:
df.isna().sum(axis=0)

week_start                0
user_id_hash              0
no_city                   0
city                      0
is_user_first_session     0
country                   0
country_count             0
region                    0
region_count              0
locale                    0
os_name                   0
user_days                 0
total_session_time        0
mean_session_time         0
median_session_time       0
user_created_timestamp    0
no_of_buys                0
sum_of_buys               0
y1                        0
y2                        0
dtype: int64

In [38]:
from sklearn.model_selection import KFold

def reg_target_encoding(train, col = "device_type", splits=5):
    """ Computes regularize mean encoding.
    Inputs:
       train: training dataframe
       
    """
    ksplit = KFold(n_splits=splits)
    for tidx, vidx in ksplit.split(train):
        mean_val = train.iloc[tidx, :].groupby(col)['y1'].mean()
        global_mean_val = train.iloc[tidx, :]['y1'].mean()
        train.loc[vidx, col + "_mean_enc"] = train.iloc[vidx, :][col].map(mean_val)
        train.loc[vidx, col + "_mean_enc"].fillna(global_mean_val, inplace=True)
    return None

In [39]:
def mean_encoding_test(test, train, col = "device_type"):
    """ Computes target enconding for test data.
    
    This is similar to how we do validation
    """
    mean_device_type = train.groupby(col)['y1'].mean()
    global_mean = train['y1'].mean()
    test[col + "_mean_enc"] = test[col].map(mean_device_type)
    test[col + "_mean_enc"].fillna(global_mean, inplace=True)
    return None

In [40]:
df.city = df.city.astype('category')
df.country = df.country.astype('category')
df.region = df.region.astype('category')
df.locale = df.locale.astype('category')
df.os_name = df.os_name.astype('category')

In [41]:
df['user_age'] = (df.week_start.dt.date - df.user_created_timestamp)\
                    .astype('timedelta64[D]')

In [42]:
df.head()

,week_start,user_id_hash,no_city,city,is_user_first_session,country,country_count,region,region_count,locale,...,user_days,total_session_time,mean_session_time,median_session_time,user_created_timestamp,no_of_buys,sum_of_buys,y1,y2,user_age
0,2018-10-01,000062e9be78f3da274fec338e78f89d12000e781967f2...,5,campbell,True,US,1,ca,1,en_US,...,1,4730609,946121.8,1176899.0,2018-10-07,0.0,0.0,False,False,-6.0
1,2018-10-01,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,douglasville,True,US,1,ga,1,en_US,...,2,403432,201716.0,201716.0,2018-10-05,0.0,0.0,False,False,-4.0
2,2018-10-01,0003f8bda56230a49445880e559b718e94ba37344494a7...,2,middlesboro,True,US,1,ky,1,en_US,...,2,150138,75069.0,75069.0,2018-10-03,0.0,0.0,False,False,-2.0
3,2018-10-01,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1,omaha,True,US,1,ne,1,en_US,...,1,0,0.0,0.0,2018-10-02,0.0,0.0,False,False,-1.0
4,2018-10-01,00059c820dca1ccadfd45361c40f68b656df8a5212b993...,1,rio de janeiro,False,BR,1,rj,1,pt-BR_BR,...,1,123131,123131.0,123131.0,2018-10-01,0.0,0.0,False,False,0.0


In [43]:
df2 = df.copy(deep=True)

In [44]:
df2.week_start = df2.week_start.apply(lambda x: x + dt.timedelta(days=7))

In [45]:
df2.head()

,week_start,user_id_hash,no_city,city,is_user_first_session,country,country_count,region,region_count,locale,...,user_days,total_session_time,mean_session_time,median_session_time,user_created_timestamp,no_of_buys,sum_of_buys,y1,y2,user_age
0,2018-10-08,000062e9be78f3da274fec338e78f89d12000e781967f2...,5,campbell,True,US,1,ca,1,en_US,...,1,4730609,946121.8,1176899.0,2018-10-07,0.0,0.0,False,False,-6.0
1,2018-10-08,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,douglasville,True,US,1,ga,1,en_US,...,2,403432,201716.0,201716.0,2018-10-05,0.0,0.0,False,False,-4.0
2,2018-10-08,0003f8bda56230a49445880e559b718e94ba37344494a7...,2,middlesboro,True,US,1,ky,1,en_US,...,2,150138,75069.0,75069.0,2018-10-03,0.0,0.0,False,False,-2.0
3,2018-10-08,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1,omaha,True,US,1,ne,1,en_US,...,1,0,0.0,0.0,2018-10-02,0.0,0.0,False,False,-1.0
4,2018-10-08,00059c820dca1ccadfd45361c40f68b656df8a5212b993...,1,rio de janeiro,False,BR,1,rj,1,pt-BR_BR,...,1,123131,123131.0,123131.0,2018-10-01,0.0,0.0,False,False,0.0


In [46]:
df2.columns

Index(['week_start', 'user_id_hash', 'no_city', 'city',
       'is_user_first_session', 'country', 'country_count', 'region',
       'region_count', 'locale', 'os_name', 'user_days', 'total_session_time',
       'mean_session_time', 'median_session_time', 'user_created_timestamp',
       'no_of_buys', 'sum_of_buys', 'y1', 'y2', 'user_age'],
      dtype='object')

In [47]:
merge_cols = ['week_start', 'user_id_hash', 'no_city', 'city',
             'is_user_first_session', 'country', 'country_count',
             'region','region_count', 'locale', 'user_days', 
             'total_session_time', 'mean_session_time',
             'median_session_time', 'no_of_buys', 'sum_of_buys']

In [48]:
df.shape

(994636, 21)

In [49]:
df = df.merge(df2[merge_cols], on=['user_id_hash', 'week_start'], how='left')

In [50]:
df.shape

(994636, 35)

In [51]:
df.head()

,week_start,user_id_hash,no_city_x,city_x,is_user_first_session_x,country_x,country_count_x,region_x,region_count_x,locale_x,...,country_count_y,region_y,region_count_y,locale_y,user_days_y,total_session_time_y,mean_session_time_y,median_session_time_y,no_of_buys_y,sum_of_buys_y
0,2018-10-01,000062e9be78f3da274fec338e78f89d12000e781967f2...,5,campbell,True,US,1,ca,1,en_US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-10-01,00026e5050a70ef12d421f75c6a5c80d0f62d37acab8bd...,2,douglasville,True,US,1,ga,1,en_US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-10-01,0003f8bda56230a49445880e559b718e94ba37344494a7...,2,middlesboro,True,US,1,ky,1,en_US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-10-01,000542d6fc6e9dcb83328d30503a7e022e5a6b4ea6357b...,1,omaha,True,US,1,ne,1,en_US,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-10-01,00059c820dca1ccadfd45361c40f68b656df8a5212b993...,1,rio de janeiro,False,BR,1,rj,1,pt-BR_BR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
df.isna().sum(axis=0)

week_start                      0
user_id_hash                    0
no_city_x                       0
city_x                          0
is_user_first_session_x         0
country_x                       0
country_count_x                 0
region_x                        0
region_count_x                  0
locale_x                        0
os_name                         0
user_days_x                     0
total_session_time_x            0
mean_session_time_x             0
median_session_time_x           0
user_created_timestamp          0
no_of_buys_x                    0
sum_of_buys_x                   0
y1                              0
y2                              0
user_age                        0
no_city_y                  677455
city_y                     677455
is_user_first_session_y    677455
country_y                  677455
country_count_y            677455
region_y                   677455
region_count_y             677455
locale_y                   677455
user_days_y   

In [53]:
df.shape

(994636, 35)

In [54]:
df = df[df.week_start != '2018-10-01']

In [55]:
df.shape

(930062, 35)

In [60]:
df.isna().sum(axis=0)

week_start                      0
user_id_hash                    0
no_city_x                       0
city_x                          0
is_user_first_session_x         0
country_x                       0
country_count_x                 0
region_x                        0
region_count_x                  0
locale_x                        0
os_name                         0
user_days_x                     0
total_session_time_x            0
mean_session_time_x             0
median_session_time_x           0
user_created_timestamp          0
no_of_buys_x                    0
sum_of_buys_x                   0
y1                              0
y2                              0
user_age                        0
no_city_y                  612881
city_y                     612881
is_user_first_session_y    612881
country_y                  612881
country_count_y            612881
region_y                   612881
region_count_y             612881
locale_y                   612881
user_days_y   

In [61]:
df.dtypes

week_start                 datetime64[ns]
user_id_hash                       object
no_city_x                           int64
city_x                           category
is_user_first_session_x              bool
country_x                        category
country_count_x                     int64
region_x                         category
region_count_x                      int64
locale_x                         category
os_name                          category
user_days_x                         int64
total_session_time_x                int64
mean_session_time_x               float64
median_session_time_x             float64
user_created_timestamp             object
no_of_buys_x                      float64
sum_of_buys_x                     float64
y1                                   bool
y2                                   bool
user_age                          float64
no_city_y                         float64
city_y                           category
is_user_first_session_y           

In [62]:
fill_cols = ['city_y', 'country_y', 'region_y', 'locale_y']

In [63]:
for c in fill_cols:
    df[c] = df[c].cat.add_categories([-99])
    df[c].fillna(-99, inplace=True)

In [66]:
fill_cols = ['no_city_y', 'is_user_first_session_y', 'country_count_y',
             'region_count_y', 'user_days_y', 'total_session_time_y',
             'mean_session_time_y', 'median_session_time_y', 'no_of_buys_y',
             'sum_of_buys_y']

In [67]:
for c in fill_cols:
    df[c].fillna(-99, inplace=True)

In [70]:
df.week_start = df.week_start.astype('category')

In [73]:
df.week_start = df.week_start.dt.date

In [74]:
df.week_start.unique()

array([datetime.date(2018, 10, 8), datetime.date(2018, 10, 15),
       datetime.date(2018, 10, 22), datetime.date(2018, 10, 29),
       datetime.date(2018, 11, 5), datetime.date(2018, 11, 12),
       datetime.date(2018, 11, 19), datetime.date(2018, 11, 26),
       datetime.date(2018, 12, 3), datetime.date(2018, 12, 10)],
      dtype=object)

In [90]:
df.reset_index(inplace=True, drop=True)

In [91]:
train = df[df.week_start < dt.date(2018, 12, 10)]
# test = df[df.week_start >= dt.date(2018, 12, 10)]

In [92]:
df[df.week_start == dt.date(2018, 12, 3)].shape

(45038, 35)

In [93]:
df[df.week_start == dt.date(2018, 12, 10)].shape

(29487, 35)

In [94]:
df[df.week_start == dt.date(2018, 12, 10)]['y1'].sum()

0

In [95]:
df[df.week_start == dt.date(2018, 12, 3)]['y1'].sum()

2110

In [96]:
df[df.week_start == dt.date(2018, 12, 3)]['y2'].sum()

2110

In [97]:
train.columns

Index(['week_start', 'user_id_hash', 'no_city_x', 'city_x',
       'is_user_first_session_x', 'country_x', 'country_count_x', 'region_x',
       'region_count_x', 'locale_x', 'os_name', 'user_days_x',
       'total_session_time_x', 'mean_session_time_x', 'median_session_time_x',
       'user_created_timestamp', 'no_of_buys_x', 'sum_of_buys_x', 'y1', 'y2',
       'user_age', 'no_city_y', 'city_y', 'is_user_first_session_y',
       'country_y', 'country_count_y', 'region_y', 'region_count_y',
       'locale_y', 'user_days_y', 'total_session_time_y',
       'mean_session_time_y', 'median_session_time_y', 'no_of_buys_y',
       'sum_of_buys_y'],
      dtype='object')

In [99]:
from warnings import filterwarnings

In [100]:
filterwarnings('ignore')

In [101]:
reg_target_encoding(train, col='country_x')

In [103]:
reg_target_encoding(train, col='week_start')

In [104]:
reg_target_encoding(train, col='region_x')

In [105]:
train.city_x = train.city_x.cat.codes
train.city_y = train.city_y.cat.codes
train.country_x = train.country_x.cat.codes
train.country_y = train.country_y.cat.codes
train.region_x = train.region_x.cat.codes
train.region_y = train.region_y.cat.codes
train.os_name = train.os_name.cat.codes
train.locale_x = train.locale_x.cat.codes
train.locale_y = train.locale_y.cat.codes

In [107]:
import numpy as np

In [108]:
train.is_user_first_session_x = train.is_user_first_session_x.astype(np.int)
train.is_user_first_session_y = train.is_user_first_session_y.astype(np.int)
train['y1'] = train['y1'].astype(np.int)
train['y2'] = train['y2'].astype(np.int)

In [109]:
train.shape

(900575, 38)

In [116]:
feature_cols = ['no_city_x', 'city_x', 'is_user_first_session_x',
                'country_x', 'country_count_x', 'region_x',
                'region_count_x', 'locale_x', 'os_name', 'user_days_x',
                'total_session_time_x', 'mean_session_time_x',
                'median_session_time_x', 'no_of_buys_x', 'sum_of_buys_x',
                'user_age', 'no_city_y', 'city_y', 'is_user_first_session_y',
                'country_y', 'country_count_y', 'region_y', 'region_count_y',
                'locale_y', 'user_days_y', 'total_session_time_y',
                'mean_session_time_y', 'median_session_time_y', 'no_of_buys_y',
                'sum_of_buys_y', 'country_x_mean_enc',
                'region_x_mean_enc']

In [119]:
train.country_x_mean_enc.mean()

0.037305791269076946

In [120]:
train.region_x_mean_enc.mean()

0.03730488873248875

In [121]:
train.country_x_mean_enc.fillna(0.0373, inplace=True)
train.region_x_mean_enc.fillna(0.0373, inplace=True)

In [123]:
x_train = train[feature_cols].values
y1_train = train[['y1']]
y2_train = train[['y2']]

In [125]:
import imblearn

In [126]:
from imblearn.over_sampling import RandomOverSampler

In [128]:
ros = RandomOverSampler(return_indices=True)

In [129]:
X_ros, y_ros, id_ros = ros.fit_sample(x_train, y1_train)

In [130]:
X_ros.shape

(1733972, 32)

In [132]:
sum(y_ros)

866986

In [133]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [2]:
xg_model1 = XGBClassifier(max_depth=5, learning_rate=0.01, n_estimators=200, n_jobs=-1)

In [135]:
xg_model1.fit(X_ros, y_ros)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [136]:
y1_hat_train = xg_model1.predict(x_train)

In [137]:
accuracy_score(y1_train, y1_hat_train)

0.8980351442134192

In [141]:
accuracy_score(y_ros, xg_model1.predict(X_ros))

0.8783377124890137

In [138]:
xg_model2 = XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=300, n_jobs=-1)

In [139]:
xg_model2.fit(x_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [140]:
accuracy_score(y1_train, xg_model2.predict(x_train))

0.9724398301085417

In [186]:
xg_model3 = XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=300, n_jobs=-1)

In [189]:
from sklearn.ensemble import RandomForestClassifier

In [192]:
rf1 = RandomForestClassifier(n_estimators=300, min_samples_leaf=10, n_jobs=-1)

In [193]:
rf1.fit(x_train, y1_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [194]:
accuracy_score(y1_train, rf1.predict(x_train))

0.9747239263803681

In [201]:
rf2 = RandomForestClassifier(n_estimators=300, min_samples_leaf=10, n_jobs=-1)

In [202]:
rf2.fit(x_train, y2_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [203]:
accuracy_score(y2_train, rf2.predict(x_train))

0.971078477639286

In [233]:
feature_cols = ['no_city_x', 'city_x', 'is_user_first_session_x',
                'country_x', 'country_count_x', 'region_x',
                'region_count_x', 'locale_x', 'user_days_x',
                'total_session_time_x', 'mean_session_time_x',
                'median_session_time_x', 'no_of_buys_x', 'sum_of_buys_x',
                'user_age']

In [234]:
rf = RandomForestClassifier(n_estimators=300, min_samples_leaf=10, n_jobs=-1)

In [235]:
rf.fit(x_train, y1_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [226]:
rf.fit(x_train, y1_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=10, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [227]:
accuracy_score(y1_train, rf.predict(x_train))

0.9747983232934514

In [236]:
accuracy_score(y1_train, rf.predict(x_train))

0.9747872192765733

In [237]:
print_feature_importance(rf)

user_age                      : 24.31%
sum_of_buys_x                 : 20.01%
mean_session_time_x           :  5.31%
median_session_time_x         :  3.79%
no_of_buys_x                  :  3.73%
no_city_x                     :  3.67%
total_session_time_x          :  3.56%
city_x                        :  2.79%
region_x                      :  1.99%
locale_x                      :  0.83%
country_x                     :  0.63%
region_count_x                :  0.38%
user_days_x                   :  0.32%
is_user_first_session_x       :  0.19%
country_count_x               :  0.01%


In [228]:
print_feature_importance(rf)

user_age                      : 23.30%
sum_of_buys_x                 : 21.80%
mean_session_time_x           :  5.08%
no_city_y                     :  4.27%
median_session_time_x         :  3.82%
no_of_buys_x                  :  3.75%
no_city_x                     :  3.64%
total_session_time_x          :  3.61%
city_x                        :  2.78%
region_x                      :  2.00%
total_session_time_y          :  1.31%
city_y                        :  1.05%
sum_of_buys_y                 :  1.00%
locale_x                      :  0.82%
country_x                     :  0.61%
region_count_x                :  0.40%
user_days_x                   :  0.32%
median_session_time_y         :  0.30%
is_user_first_session_x       :  0.22%
mean_session_time_y           :  0.21%
no_of_buys_y                  :  0.04%
country_count_x               :  0.01%


In [229]:
xg = XGBClassifier(max_depth=7, learning_rate=0.1, n_estimators=300, n_jobs=-1)

In [230]:
xg.fit(x_train, y1_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=7, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=-1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [231]:
accuracy_score(y1_train, xg.predict(x_train))

0.9724398301085417

In [232]:
print_feature_importance(xg)

mean_session_time_x           :  8.48%
no_city_y                     :  7.88%
city_x                        :  7.78%
median_session_time_x         :  5.80%
no_of_buys_x                  :  5.43%
region_x                      :  4.74%
no_city_x                     :  4.16%
locale_x                      :  3.95%
user_age                      :  3.77%
total_session_time_y          :  3.65%
total_session_time_x          :  3.33%
sum_of_buys_x                 :  3.06%
city_y                        :  3.04%
sum_of_buys_y                 :  2.04%
country_x                     :  1.65%
region_count_x                :  1.11%
user_days_x                   :  0.85%
median_session_time_y         :  0.84%
country_count_x               :  0.27%
mean_session_time_y           :  0.18%
is_user_first_session_x       :  0.08%
no_of_buys_y                  :  0.07%


In [197]:
def print_feature_importance(model):
    feature_importance = dict(zip(feature_cols, model.feature_importances_))
    for name, importance in sorted(feature_importance.items(), key=lambda x: x[1],\
                                   reverse=True):
        print(f"{name:<30}: {importance:>6.2%}")

In [204]:
print_feature_importance(rf2)

no_of_buys_x                  : 22.87%
sum_of_buys_x                 : 22.39%
sum_of_buys_y                 :  5.37%
total_session_time_x          :  5.10%
no_of_buys_y                  :  4.92%
user_age                      :  3.89%
mean_session_time_x           :  3.80%
median_session_time_x         :  3.57%
user_days_x                   :  3.55%
no_city_x                     :  3.34%
region_x_mean_enc             :  2.79%
city_x                        :  2.67%
country_x_mean_enc            :  2.02%
region_x                      :  1.90%
total_session_time_y          :  1.58%
median_session_time_y         :  1.53%
mean_session_time_y           :  1.52%
city_y                        :  1.32%
no_city_y                     :  1.00%
region_y                      :  0.95%
locale_x                      :  0.82%
country_x                     :  0.63%
user_days_y                   :  0.56%
locale_y                      :  0.39%
region_count_x                :  0.37%
country_y                

In [200]:
print_feature_importance(xg_model2)

total_session_time_x          :  8.48%
user_age                      :  7.88%
city_x                        :  7.78%
region_x_mean_enc             :  7.13%
country_x_mean_enc            :  5.91%
mean_session_time_x           :  5.80%
median_session_time_x         :  5.43%
region_x                      :  4.74%
no_city_x                     :  4.16%
locale_x                      :  3.95%
sum_of_buys_x                 :  3.77%
city_y                        :  3.65%
user_days_x                   :  3.33%
total_session_time_y          :  3.10%
no_of_buys_x                  :  3.06%
no_city_y                     :  3.04%
sum_of_buys_y                 :  2.75%
mean_session_time_y           :  2.41%
median_session_time_y         :  2.30%
no_of_buys_y                  :  2.07%
region_y                      :  2.04%
country_x                     :  1.65%
region_count_x                :  1.11%
user_days_y                   :  0.87%
os_name                       :  0.85%
country_y                

In [199]:
print_feature_importance(xg_model1)

total_session_time_x          :  8.48%
city_x                        :  7.95%
user_age                      :  7.58%
region_x_mean_enc             :  7.29%
mean_session_time_x           :  5.92%
country_x_mean_enc            :  5.61%
region_x                      :  4.87%
median_session_time_x         :  4.84%
no_city_x                     :  4.48%
no_of_buys_x                  :  4.43%
locale_x                      :  4.05%
user_days_x                   :  3.51%
city_y                        :  3.42%
total_session_time_y          :  3.12%
sum_of_buys_x                 :  3.06%
no_city_y                     :  2.78%
mean_session_time_y           :  2.60%
no_of_buys_y                  :  2.40%
median_session_time_y         :  2.25%
sum_of_buys_y                 :  2.07%
region_y                      :  1.99%
country_x                     :  1.66%
region_count_x                :  1.06%
user_days_y                   :  0.94%
region_count_y                :  0.74%
country_y                

In [223]:
feature_cols

['no_city_x',
 'city_x',
 'is_user_first_session_x',
 'country_x',
 'country_count_x',
 'region_x',
 'region_count_x',
 'locale_x',
 'os_name',
 'user_days_x',
 'total_session_time_x',
 'mean_session_time_x',
 'median_session_time_x',
 'no_of_buys_x',
 'sum_of_buys_x',
 'user_age',
 'no_city_y',
 'city_y',
 'is_user_first_session_y',
 'country_y',
 'country_count_y',
 'region_y',
 'region_count_y',
 'locale_y',
 'user_days_y',
 'total_session_time_y',
 'mean_session_time_y',
 'median_session_time_y',
 'no_of_buys_y',
 'sum_of_buys_y',
 'country_x_mean_enc',
 'region_x_mean_enc']

In [198]:
print_feature_importance(rf1)

no_of_buys_x                  : 23.33%
sum_of_buys_x                 : 21.09%
total_session_time_x          :  5.20%
sum_of_buys_y                 :  4.89%
user_age                      :  4.35%
no_of_buys_y                  :  4.18%
mean_session_time_x           :  4.00%
no_city_x                     :  3.97%
median_session_time_x         :  3.67%
user_days_x                   :  3.48%
region_x_mean_enc             :  2.92%
city_x                        :  2.77%
country_x_mean_enc            :  2.05%
region_x                      :  1.99%
total_session_time_y          :  1.60%
mean_session_time_y           :  1.59%
median_session_time_y         :  1.47%
city_y                        :  1.31%
no_city_y                     :  1.07%
region_y                      :  1.02%
locale_x                      :  0.84%
country_x                     :  0.61%
user_days_y                   :  0.53%
region_count_x                :  0.41%
locale_y                      :  0.40%
os_name                  

In [188]:
y2_train.isna().sum()

y2    0
dtype: int64

In [142]:
import pickle

In [185]:
pickle.dump(xg_model2, open("normal_y1", "wb"))

In [143]:
pickle.dump(xg_model1, open("oversampled_y1", "wb"))

In [145]:
submission = pd.read_csv('../sample_submission_2.csv')
req_uids = set(submission.user_id_hash)

In [147]:
dates = df.week_start.unique()
dates.sort()

In [148]:
dates

array([datetime.date(2018, 10, 8), datetime.date(2018, 10, 15),
       datetime.date(2018, 10, 22), datetime.date(2018, 10, 29),
       datetime.date(2018, 11, 5), datetime.date(2018, 11, 12),
       datetime.date(2018, 11, 19), datetime.date(2018, 11, 26),
       datetime.date(2018, 12, 3), datetime.date(2018, 12, 10)],
      dtype=object)

In [149]:
df.city_x = df.city_x.cat.codes
df.city_y = df.city_y.cat.codes
df.country_x = df.country_x.cat.codes
df.country_y = df.country_y.cat.codes
df.region_x = df.region_x.cat.codes
df.region_y = df.region_y.cat.codes
df.os_name = df.os_name.cat.codes
df.locale_x = df.locale_x.cat.codes
df.locale_y = df.locale_y.cat.codes

In [150]:
feature_cols

['no_city_x',
 'city_x',
 'is_user_first_session_x',
 'country_x',
 'country_count_x',
 'region_x',
 'region_count_x',
 'locale_x',
 'os_name',
 'user_days_x',
 'total_session_time_x',
 'mean_session_time_x',
 'median_session_time_x',
 'no_of_buys_x',
 'sum_of_buys_x',
 'user_age',
 'no_city_y',
 'city_y',
 'is_user_first_session_y',
 'country_y',
 'country_count_y',
 'region_y',
 'region_count_y',
 'locale_y',
 'user_days_y',
 'total_session_time_y',
 'mean_session_time_y',
 'median_session_time_y',
 'no_of_buys_y',
 'sum_of_buys_y',
 'country_x_mean_enc',
 'region_x_mean_enc']

In [152]:
reg_target_encoding(df, col='country_x')

In [153]:
reg_target_encoding(df, col='region_x')

In [157]:
df.country_x_mean_enc.fillna(0.0373, inplace=True)
df.region_x_mean_enc.fillna(0.0373, inplace=True)

In [160]:
dates.sort()

In [164]:
dates[-2:]

array([datetime.date(2018, 12, 3), datetime.date(2018, 12, 10)],
      dtype=object)

In [215]:
req_uids = set(submission.user_id_hash)
run_ids = req_uids
predict_df = pd.DataFrame(columns=['user_id_hash',
                                   'user_purchase_binary_7_days',
                                   'user_purchase_binary_14_days'])

for dts in reversed(dates[-3:]):
    t = df.loc[(df.user_id_hash.isin(run_ids)) &\
                     (df.week_start == dts), feature_cols].values
    cur_uids = df.loc[(df.user_id_hash.isin(run_ids)) &\
                     (df.week_start == dts), ['user_id_hash']].values

    pp1 = rf1.predict_proba(t)
    pp2 = rf2.predict_proba(t)

    print(f'{dts}: {pp1[:, 1].shape}')
    temp = pd.DataFrame(data={'user_id_hash': cur_uids.reshape(-1),
                              'user_purchase_binary_7_days': pp1[:,1],
                              'user_purchase_binary_14_days': pp2[:,1]})
    predict_df = pd.concat([predict_df, temp])
    run_ids = run_ids - set(cur_uids.reshape(-1))
    print(len(list(run_ids)))

2018-12-10: (16059,)
296509
2018-12-03: (12294,)
284215
2018-11-26: (27339,)
256876


In [216]:
predict_df.shape

(55692, 3)

In [217]:
left_out = list(set(submission.user_id_hash.unique()) -\
                set(predict_df.user_id_hash.unique()))

In [218]:
predict_df.user_purchase_binary_7_days.median()

0.0057208887906412716

In [219]:
predict_df.user_purchase_binary_14_days.median()

0.008188511372120524

In [220]:
temp = pd.DataFrame(data={'user_id_hash': left_out,
                          'user_purchase_binary_7_days': 0.005 * np.ones(len(left_out)),
                          'user_purchase_binary_14_days': 0.008 * np.ones(len(left_out))})

In [221]:
predict_df = pd.concat([predict_df, temp])

In [222]:
predict_df.to_csv('../9_sub.csv', index=False, float_format='%.3f')